In [1]:
import re
import os
import numpy as np
from progress.bar import Bar
from tqdm import tqdm


In [62]:

def writeRadius(r1,r2):
    lpMesh = open("loop.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()

    reg1 = r"(%r1)"
    reg2 = r"(%r2)"

    sub1 = str(r1)
    sub2 = str(r2)

    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)

    try:
        lpTemp = open("loopTemp.geo", "x")
    except:
        os.remove("loopTemp.geo")
        lpTemp = open("loopTemp.geo","x")

    lpTemp.write(loopMesh)
    lpTemp.close()


    # write surface mesh
    lpMesh = open("loopFIELD.geo", "r")
    loopMesh = lpMesh.read()
    lpMesh.close()
    reg1 = r"(%r1)"
    reg2 = r"(%r2)"
    sub1 = str(r1)
    sub2 = str(r2)
    loopMesh = re.sub(reg1,sub1,loopMesh,0,re.MULTILINE)
    loopMesh = re.sub(reg2,sub2,loopMesh,0,re.MULTILINE)
    try:
        lpTemp = open("loopFIELDTemp.geo", "x")
    except:
        os.remove("loopFIELDTemp.geo")
        lpTemp = open("loopFIELDTemp.geo","x")
    lpTemp.write(loopMesh)
    lpTemp.close()

def runGMSH():
    print(f"{os.getpid()}: Meshing geometry\n")
    # os.system("gmsh loopFIELDTemp.geo -nopopup -0 -v 1")
    # os.system("gmsh loopTemp.geo -nopopup -0 -v 1")
    os.popen("gmsh loopFIELDTemp.geo -nopopup -0 -v 1").read()
    os.popen("gmsh loopTemp.geo -nopopup -0 -v 1").read()
    try:
        os.system("rm *.geo_unrolled")
        os.remove("loopFIELDTemp.geo")
        os.remove("loopTemp.geo")
    except:
        print("nope")
    print(f"{os.getpid()}: Done meshing\n")


def runTTH():
    print(f"{os.getpid()}: Running tth settings_washer.json >/dev/null 2>&1")
    os.system("tth settings_washer.json >/dev/null 2>&1")
    print(f"{os.getpid()}: Done\n")


def getCurrent():
    iFile = open("./output/i.txt")
    it = iFile.read()
    iFile.close()
    reg = r"fone fone .+ (\d*\.\d*e-\d*)"
    i_s = re.findall(reg,it,re.MULTILINE)[0]
    return i_s
    
def runNoiseEx(b_path,current):
    path = "/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/cmake-build-debug/nex"
    print(f"{os.getpid()}: Running "+path+" "+b_path+" "+current)
    v = os.popen(path+" "+b_path+" "+current).read()
    print(f"{os.getpid()}: Done\n")
    v = v.split("\n")
    return v


def runSim():
    # First we change loop radius
    r2s = np.linspace(0.1e-6, 2e-6, 10)

# writeRadius(15e-6, 0.2e-6)

# runGMSH()

# runTTH()

# i = getCurrent()

# runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk",i)


# runSim()

## Varying D and keeping R fixed

In [4]:

SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)

r1 = 15e-6
r2s = np.linspace(0.09e-6, 1.5e-6, 30)


try:
    os.remove("resFIX_R_2.csv")
except:
    pass

res = open("resFIX_R_2.csv","w")
res.write("R,D,Analytic,Numerical\n")

for r2 in tqdm(r2s):
    writeRadius(r1, r2)
    runGMSH()
    runTTH()
    i = getCurrent()[0]
    msfn_numerical = runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk",i)
    msfn_analytical = Analytic(r1,r2*2)
    # write result to file
    res.write(f"{r1},{2*r2},{msfn_analytical},{msfn_numerical}\n")



res.close()



  0%|          | 0/30 [00:00<?, ?it/s]

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1
Done

Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 4.1148850270e-02


  3%|▎         | 1/30 [01:56<56:04, 116.01s/it]

Done with tth!

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1
Done

Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 7.4224615707e-02


  7%|▋         | 2/30 [02:59<39:38, 84.96s/it] 

...

Done

Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 3.0510806094e-01
Done with tth!

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1


 97%|█████████▋| 29/30 [09:13<00:07,  7.08s/it]

Done

Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 3.0966398147e-01
Done with tth!

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1


100%|██████████| 30/30 [09:20<00:00, 18.68s/it]

Done

Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 3.1498253779e-01
Done with tth!



## FIX D VARY R

In [3]:
SPIN_DENS = 5e17
MU_B = 9.2740100783e-24
PHI_0 = 2.067833848e-15
MU_0 = 1.25663706212e-6 	
Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)

r1s = np.linspace(15e-6, 30e-6, 5)
r2 = 0.2e-6


try:
    os.remove("resFIX_D_AreaErr.csv")
except:
    pass

res = open("resFIX_D_AreaErr.csv","w")
res.write("R,D,Analytic,Numerical,Analytic Area, Numerical Area\n")

for r1 in tqdm(r1s):
    writeRadius(r1, r2)
    runGMSH()
    runTTH()
    i = getCurrent()
    print(i)
    msfn_numerical = runNoiseEx("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk",i)
    numerical_area = msfn_numerical[1]
    msfn_numerical = msfn_numerical[0]
    msfn_analytical = Analytic(r1,r2*2)
    analytic_area = 4*np.pi**2*r1*r2
    # write result to file
    res.write(f"{r1},{2*r2},{msfn_analytical},{msfn_numerical},{analytic_area},{numerical_area}\n")



res.close()

  0%|          | 0/5 [00:00<?, ?it/s]

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1
Done

1.0943227870e-01
Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 1.0943227870e-01


 20%|██        | 1/5 [00:37<02:29, 37.39s/it]

Done

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1
Done

8.5015196732e-02
Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 8.5015196732e-02


 40%|████      | 2/5 [01:30<02:20, 46.68s/it]

...

Done

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1
Done

5.8251250161e-02
Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 5.8251250161e-02


 80%|████████  | 4/5 [04:09<01:09, 69.82s/it]

Done

Meshing geometry
Done meshing

Running tth settings_washer.json >/dev/null 2>&1
Done

5.0158929349e-02
Running ../../cmake-build-debug/nex /home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/output/loop_test.vtk 5.0158929349e-02


100%|██████████| 5/5 [05:58<00:00, 71.73s/it]

Done



## An attempt to multithread my testing

In [56]:
os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/")

In [72]:

def runSim(folderPath,r1,r2):
    try:
        os.mkdir(folderPath)
    except:
        os.system(f"rm -rf {folderPath}")
        os.mkdir(folderPath)
    os.chdir(folderPath)
    os.system("cp ../loop.geo ./")
    os.system("cp ../loopFIELD.geo ./")
    os.system("cp ../settings_washer.json ./")
    SPIN_DENS = 5e17
    MU_B = 9.2740100783e-24
    PHI_0 = 2.067833848e-15
    MU_0 = 1.25663706212e-6 	
    Analytic = lambda R, D : 2/3*MU_0**2*MU_B**2*SPIN_DENS*R/D*1/(PHI_0**2)
    
    writeRadius(r1, r2)
    runGMSH()
    runTTH()
    i = getCurrent()
    msfn_numerical = runNoiseEx(f"/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/{folderPath}/output/loop_test.vtk",i)
    print(msfn_numerical)
    numerical_area = msfn_numerical[1]
    msfn_numerical = msfn_numerical[0]
    msfn_analytical = Analytic(r1,r2*2)
    analytic_area = 4*np.pi**2*r1*r2
    # write result to file
    val = f"{r1},{2*r2},{msfn_analytical},{msfn_numerical},{analytic_area},{numerical_area}\n"
    
    try:
        os.remove("resPARR_SIM.csv")
    except:
        pass

    res = open("resPARR_SIM.csv","w")
    res.write("R,D,Analytic,Numerical,Analytic Area, Numerical Area\n")
    res.write(val)
    res.close()
    os.chdir("/home/paul/Skripsie/Code/SkripsieCode/cpp/noiseEX/testfiles/aVSn/")

    return val

    


In [84]:
from multiprocessing import Process
from time import sleep

In [91]:
r1s = np.linspace(15e-6, 30e-6, 50)
r2 = 0.2e-6
pl = []
count = 0
for r1 in r1s:
    count = count + 1
    path = f"./sim{count}"
    pl.append(Process(target=runSim,args=(path,r1,r2,)))

t = pl[2]
pl[2] = pl[0]
pl[0] = t

for p in pl:
    p.start()

not_done = True
while not_done:
    not_done = False
    for p in pl:
        if p.is_alive() == True:
            not_done = True
    sleep(0.5)

print("\n\n Sim complete")
    

65574: Meshing geometry
65576: Meshing geometry
65580: Meshing geometry
65586: Meshing geometry


65596: Meshing geometry
65608: Meshing geometry
65632: Meshing geometry
65623: Meshing geometry
65661: Meshing geometry
65654: Meshing geometry
65674: Meshing geometry65680: Meshing geometry
65695: Meshing geometry

65675: Meshing geometry65699: Meshing geometry65683: Meshing geometry
65726: Meshing geometry

65745: Meshing geometry

65780: Meshing geometry
65758: Meshing geometry65793: Meshing geometry

65817: Meshing geometry
65838: Meshing geometry65855: Meshing geometry

65900: Meshing geometry65874: Meshing geometry

65908: Meshing geometry65957: Meshing geometry65937: Meshing geometry


65974: Meshing geometry
66029: Meshing geometry
65992: Meshing geometry
66034: Meshing geometry66046: Meshing geometry

66058: Meshing geometry
66102: Meshing geometry66094: Meshing geometry
66124: Meshing geometry

66138: Meshing geometry
66151: Meshing geometry
66177: Meshing geometry66183: Meshing geometry66188: Meshing geometry


66211: Meshing geometry66199: Meshing geometry66182: Meshing ge

In [87]:
for p in pl:
    p.kill()

In [90]:
for p in pl:
    print(p.is_alive())

False
False
False
False
False
